In [1]:
# Email Re: BL Conquesting - Pier 1, Friday, January 24, 2020 at 12:59 PM

import pandas as pd
import numpy as np
import datetime
import os

os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2019_Q4/Planner_Request/Ramiro_TA_request_20200124'

In [2]:
File_Excel=pd.ExcelFile("/home/jian/Projects/Big_Lots/New_TA/TA_created_in_201906/final_output_20190718/BL_final_TA_updated_JL_2019-07-18.xlsx")
File_Excel.sheet_names

['unique_zips_full_footprint',
 'view_by_TA',
 'view_by_store',
 'view_for_Tableau']

In [3]:
df_old_view_by_store=File_Excel.parse("view_by_store",dtype=str)
print(len(df_old_view_by_store))
df_old_view_by_zip_store=File_Excel.parse("view_for_Tableau",dtype=str)
# df_old_view_by_TA=File_Excel.parse("view_for_Tableau",dtype=str)
# To aggregate from new


1408


In [4]:
df_old_view_by_store.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,DMA,CTY,all_trans_P_zips,all_trans_S_zips,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,{'DALLAS-FT. WORTH'},{'DALLAS'},"['75040', '75044', '75048', '75098']","['75089', '75042', '75043', '75041', '75088', ...",[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,{'DALLAS-FT. WORTH'},"{'HUNT', 'KAUFMAN', 'ROCKWALL'}","['75160', '75126', '75161']","['75142', '75169', '75474', '75143', '75114', ...",[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [5]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                         dtype=str,sep="|",usecols=['location_id','latitude_meas','longitude_meas'])
store_list=store_list[store_list['location_id']!="6990"]
store_list=store_list[store_list['location_id']!="145"]
store_list['latitude_meas']=store_list['latitude_meas'].astype(float)
store_list['longitude_meas']=store_list['longitude_meas'].astype(float)
df_new_stores=store_list[~store_list['location_id'].isin(df_old_view_by_store['location_id'].tolist())]
print(df_new_stores.shape)
df_new_stores['store_location']=df_new_stores[['latitude_meas','longitude_meas']].values.tolist()
dict_new_stores=df_new_stores.set_index("location_id").to_dict()['store_location']

(30, 3)


/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [6]:
import json
from haversine import haversine
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json","r"))

df_new_store_zip=pd.DataFrame()
for location_id,store_center in dict_new_stores.items():
    zip_list_store=[]
    for z, z_c in zip_centers.items():
        if haversine(store_center,z_c,unit="mi")<=10:
            zip_list_store.append(z)
    df_new_store_zip=df_new_store_zip.append(pd.DataFrame({"location_id":[location_id]*len(zip_list_store),"zip_cd":zip_list_store}))


df_new_store_zip['zip_type']="zips_10"

In [7]:
df_new_store_zip.head(2)

,location_id,zip_cd,zip_type
0,4679,48919,zips_10
1,4679,48912,zips_10


In [8]:
closed_stores=[x for x in df_old_view_by_store['location_id'].tolist() if x not in store_list['location_id'].tolist()]
print(len(closed_stores))
# closed_stores

18


In [9]:
df_new_view_by_store_0=df_old_view_by_store[~df_old_view_by_store['location_id'].isin(closed_stores)]
print(len(df_new_view_by_store_0))
df_new_view_by_store_0=df_new_view_by_store_0[['TA_num','ta_name','location_id','address_line_1','address_line_2',
                                               'zip_cd','city_nm','state_nm','latitude_meas','longitude_meas',
                                               'zips_in_10','trans_P_zips_70_within_TA','trans_S_zips_70_within_TA']]
df_new_view_by_store_0['TA_num']=df_new_view_by_store_0['TA_num'].astype(int)
df_new_view_by_store_0.head(2)

1390


,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [10]:
df_new_view_by_store_0.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,1,multiple_1,1949,3178 LAVON DR,NaN,75040,GARLAND,TX,32.945015,-96.619194,[],"['75040', '75098', '75044', '75048']","['75088', '75042', '75082', '75043', '75089', ..."
1,1,multiple_1,1255,1418 W MOORE AVE,NaN,75160,TERRELL,TX,32.738772,-96.299877,[],"['75126', '75161', '75160']","['75142', '75169', '75150', '75117', '75143', ..."


In [11]:
df_new_view_by_store_1=df_new_store_zip.groupby("location_id")['zip_cd'].apply(list).to_frame().reset_index()
df_new_view_by_store_1['zip_cd']=df_new_view_by_store_1['zip_cd'].astype(str)
df_new_view_by_store_1=df_new_view_by_store_1.rename(columns={"zip_cd":"zips_in_10"})
df_new_view_by_store_1.head(2)

store_list_copy=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                         dtype=str,sep="|")
                         
store_list_copy=store_list_copy[[x for x in df_new_view_by_store_0.columns.tolist() if x in store_list_copy.columns.tolist()]]
store_list_copy['latitude_meas']=store_list_copy['latitude_meas'].astype(float)
store_list_copy['longitude_meas']=store_list_copy['longitude_meas'].astype(float)
df_new_view_by_store_1=pd.merge(df_new_view_by_store_1,store_list_copy,on="location_id",how="left")
df_new_view_by_store_1['ta_name']=["new_"+str(i+1) for i in range(len(df_new_view_by_store_1))]
df_new_view_by_store_1['TA_num']=[str(i+1+max(df_new_view_by_store_0['TA_num'])) for i in range(len(df_new_view_by_store_1))]
df_new_view_by_store_1['trans_P_zips_70_within_TA']="[]"
df_new_view_by_store_1['trans_S_zips_70_within_TA']="[]"
df_new_view_by_store_1=df_new_view_by_store_1[df_new_view_by_store_0.columns.tolist()]
df_new_view_by_store_1.head(2)

,TA_num,ta_name,location_id,address_line_1,address_line_2,zip_cd,city_nm,state_nm,latitude_meas,longitude_meas,zips_in_10,trans_P_zips_70_within_TA,trans_S_zips_70_within_TA
0,707,new_1,4678,3501 E LINCOLNWAY,NaN,82001-6302,CHEYENNE,WY,41.137076,-104.767025,"['82001', '82009', '82007', '82003', '82005', ...",[],[]
1,708,new_2,4679,2020 GRAND RIVER AVE,NaN,48864-1776,OKEMOS,MI,42.723314,-84.424187,"['48919', '48912', '48930', '48933', '48937', ...",[],[]


In [12]:
df_new_view_by_store=df_new_view_by_store_0.append(df_new_view_by_store_1)

del df_new_view_by_store_0
del df_new_view_by_store_1

In [13]:
df_new_view_by_store['zips_in_10']=df_new_view_by_store['zips_in_10'].astype(str)
df_new_view_by_store['trans_P_zips_70_within_TA']=df_new_view_by_store['trans_P_zips_70_within_TA'].astype(str)
df_new_view_by_store['trans_S_zips_70_within_TA']=df_new_view_by_store['trans_S_zips_70_within_TA'].astype(str)


In [14]:
df_old_view_by_zip_store.head(2)

,TA_num,location_id,ta_name,zip_cd,zip_type,latitude_meas,longitude_meas
0,1,1949,multiple_1,75040,trans_P,32.945015,-96.619194
1,1,1949,multiple_1,75098,trans_P,32.945015,-96.619194


In [15]:
df_new_view_by_zip_store=pd.DataFrame()

for ind,row in df_new_view_by_store.iterrows():
    df_P=pd.DataFrame({"zip_cd":eval(row['trans_P_zips_70_within_TA'])})
    df_S=pd.DataFrame({"zip_cd":eval(row['trans_S_zips_70_within_TA'])})
    df_10=pd.DataFrame({"zip_cd":eval(row['zips_in_10'])})
    df_P['location_id']=row['location_id']
    df_P['zip_type']="trans_P"
    
    df_S['location_id']=row['location_id']
    df_S['zip_type']="trans_S"
    
    df_10['location_id']=row['location_id']
    df_10['zip_type']="zips_10"
    
    df_zips_by_store=df_P.append(df_S).append(df_10)
    
    df_zips_by_store['TA_num']=row['TA_num']
    df_zips_by_store['location_id']=row['location_id']
    df_zips_by_store['ta_name']=row['ta_name']
    df_zips_by_store['latitude_meas']=row['latitude_meas']
    df_zips_by_store['longitude_meas']=row['longitude_meas']
    
    df_new_view_by_zip_store=df_new_view_by_zip_store.append(df_zips_by_store)
    
df_new_view_by_zip_store=df_new_view_by_zip_store[df_old_view_by_zip_store.columns.tolist()]
print(len(df_new_view_by_zip_store))

23634


In [16]:
df_new_view_by_zip_store_copy=df_new_view_by_zip_store.copy()
df_new_view_by_zip_store_copy=df_new_view_by_zip_store_copy.sort_values(['TA_num','ta_name','zip_type'])
df_new_view_by_zip_store_copy=df_new_view_by_zip_store_copy.drop_duplicates(['TA_num','ta_name','zip_cd'])


In [17]:
df_new_view_by_zip_store_copy.tail(5)

,TA_num,location_id,ta_name,zip_cd,zip_type,latitude_meas,longitude_meas
21,736,5411,new_30,23661,zips_10,37.0396,-76.4059
22,736,5411,new_30,23666,zips_10,37.0396,-76.4059
23,736,5411,new_30,23668,zips_10,37.0396,-76.4059
24,736,5411,new_30,23681,zips_10,37.0396,-76.4059
25,736,5411,new_30,23612,zips_10,37.0396,-76.4059


In [18]:
df_new_view_by_TA=df_new_view_by_zip_store.groupby(['TA_num','ta_name'])['location_id','zip_cd'].agg(set).reset_index()
df_new_view_by_TA['store_count']=df_new_view_by_TA['location_id'].apply(len)
df_new_view_by_TA['zip_count']=df_new_view_by_TA['zip_cd'].apply(len)

df_new_view_by_TA_2=df_new_view_by_zip_store_copy.groupby(['TA_num','ta_name','zip_type'])['zip_cd'].agg(set).to_frame().reset_index()
df_new_view_by_TA_P=df_new_view_by_TA_2[df_new_view_by_TA_2['zip_type']=="trans_P"][['TA_num','ta_name','zip_cd']].rename(columns={"zip_cd":"list_P_zips"})
df_new_view_by_TA_S=df_new_view_by_TA_2[df_new_view_by_TA_2['zip_type']=="trans_S"][['TA_num','ta_name','zip_cd']].rename(columns={"zip_cd":"list_S_zips"})
df_new_view_by_TA_10=df_new_view_by_TA_2[df_new_view_by_TA_2['zip_type']=="zips_10"][['TA_num','ta_name','zip_cd']].rename(columns={"zip_cd":"list_10_zips"})
del df_new_view_by_TA_2

df_new_view_by_TA=pd.merge(df_new_view_by_TA,df_new_view_by_TA_P,on=['TA_num','ta_name'],how="left")
df_new_view_by_TA=pd.merge(df_new_view_by_TA,df_new_view_by_TA_S,on=['TA_num','ta_name'],how="left")
df_new_view_by_TA=pd.merge(df_new_view_by_TA,df_new_view_by_TA_10,on=['TA_num','ta_name'],how="left")

df_new_view_by_TA['list_P_zips']=df_new_view_by_TA['list_P_zips'].astype(str)
df_new_view_by_TA['list_P_zips']=df_new_view_by_TA['list_P_zips'].fillna("[]").replace("nan","[]")

df_new_view_by_TA['list_S_zips']=df_new_view_by_TA['list_S_zips'].astype(str)
df_new_view_by_TA['list_S_zips']=df_new_view_by_TA['list_S_zips'].fillna("[]").replace("nan","[]")

df_new_view_by_TA['list_10_zips']=df_new_view_by_TA['list_10_zips'].astype(str)
df_new_view_by_TA['list_10_zips']=df_new_view_by_TA['list_10_zips'].fillna("[]").replace("nan","[]")


df_new_view_by_TA.head(5)

,TA_num,ta_name,location_id,zip_cd,store_count,zip_count,list_P_zips,list_S_zips,list_10_zips
0,1,multiple_1,"{1624, 1179, 4140, 1860, 4134, 1255, 1027, 409...","{75180, 75409, 75474, 76078, 75169, 75052, 750...",21,187,"{'75228', '75161', '75043', '75074', '75056', ...","{'75180', '75409', '75474', '76078', '75169', ...",[]
1,2,multiple_2,"{1668, 77, 410, 1379, 5092, 122, 1666, 1079, 5...","{43319, 43334, 43358, 43064, 44903, 43318, 432...",16,120,"{'43035', '43040', '43215', '43229', '43762', ...","{'43045', '43319', '43222', '43326', '43749', ...",[]
2,3,multiple_3,"{353, 5255, 1205, 310, 1797, 5325, 1845, 1078,...","{37051, 37334, 37221, 37215, 37083, 37330, 370...",16,122,"{'37221', '37205', '37167', '37036', '37015', ...","{'37135', '37051', '37030', '37101', '37048', ...",[]
3,4,multiple_4,"{1643, 315, 5119, 1588, 5276, 5237, 1138, 1521...","{28144, 28032, 28092, 28682, 27295, 29210, 282...",16,125,"{'28144', '28016', '28104', '28277', '28270', ...","{'29706', '28032', '28164', '28088', '29704', ...",[]
4,5,multiple_5,"{4321, 4051, 4291, 4688, 4045, 1995, 4169, 425...","{92336, 92509, 92501, 91761, 92571, 91784, 917...",15,89,"{'92410', '91752', '92336', '92509', '92324', ...","{'92301', '92883', '92354', '92860', '92807', ...",[]


In [19]:
df_unique_zip=df_new_view_by_zip_store.copy()
df_unique_zip=df_unique_zip.sort_values(['zip_type','zip_cd'])
df_unique_zip=df_unique_zip[['zip_type','zip_cd']].drop_duplicates("zip_cd")
print(df_unique_zip.shape)

(13324, 2)


In [20]:
store_list_full=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20200101-135601-098.txt",
                         dtype=str,sep="|")
store_list_full.shape

(1422, 13)

In [21]:
writer=pd.ExcelWriter("./BL_zips_in_TA_2020_updated_30_new_stores_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_unique_zip.to_excel(writer,"unique_zips",index=False)
df_new_view_by_TA.to_excel(writer,"view_by_TA",index=False)
df_new_view_by_store.to_excel(writer,"view_by_store",index=False)
df_new_view_by_zip_store.to_excel(writer,"view_by_store_and_zip",index=False)
store_list_full.to_excel(writer,"store_list_full_202001",index=False)
writer.save()